## Introduction 

In [2]:
import pandas as pd 
import numpy as np 
import zipfile  
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline

from sklearn.preprocessing import StandardScaler  


## Load Datasets 

* Train Dataset

In [3]:
import zipfile

zip_path = r"C:\Users\wanji\Desktop\WIDS Challenge\widsdatathon2025.zip"

# List files inside the ZIP
with zipfile.ZipFile(zip_path, 'r') as z:
    print(z.namelist())  # This will list all files inside the ZIP


['Data Dictionary.xlsx', 'SAMPLE_SUBMISSION.xlsx', 'TEST/TEST_CATEGORICAL.xlsx', 'TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv', 'TEST/TEST_QUANTITATIVE_METADATA.xlsx', 'TRAIN/TRAINING_SOLUTIONS.xlsx', 'TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx', 'TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv', 'TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx']


In [4]:
file_to_read = "TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv"

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(file_to_read) as f:
        train_data = pd.read_csv(f)

train_data.head()  # Display first few rows



,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,70z8Q2xdTXM3,0.093473,0.146902,0.067893,0.015141,0.070221,0.063997,0.055382,-0.035335,0.068583,...,0.003404,-0.010359,-0.050968,-0.014365,0.128066,0.112646,-0.058980,0.028228,0.133582,0.143372
1,WHWymJu6zNZi,0.029580,0.179323,0.112933,0.038291,0.104899,0.064250,0.008488,0.077505,-0.004750,...,-0.008409,-0.008479,0.020891,0.017754,0.094040,0.035141,0.032537,0.075007,0.115350,0.138200
2,4PAQp1M6EyAo,-0.051580,0.139734,0.068295,0.046991,0.111085,0.026978,0.151377,0.021198,0.083721,...,0.053245,-0.028003,0.028773,0.024556,0.166343,0.058925,0.035485,0.063661,0.042862,0.162162
3,obEacy4Of68I,0.016273,0.204702,0.115980,0.043103,0.056431,0.057615,0.055773,0.075030,0.001033,...,-0.023918,-0.005356,0.018607,0.016193,0.072955,0.130135,0.056120,0.084784,0.114148,0.190584
4,s7WzzDcmDOhF,0.065771,0.098714,0.097604,0.112988,0.071139,0.085607,0.019392,-0.036403,-0.020375,...,0.066439,-0.076680,-0.047530,-0.031443,0.221213,0.007343,0.005763,0.083820,0.079582,0.067269


* Test Dataset 

In [5]:
file_to_read = "TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv"

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(file_to_read) as f:
        test_data = pd.read_csv(f)

test_data.head()        

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,Cfwaf5FX7jWK,0.548480,0.713607,0.557319,0.524369,0.693364,0.770032,0.724406,0.390118,0.547912,...,0.080423,-0.054581,-0.088163,-0.028574,0.444847,0.350149,-0.012601,0.665750,0.560565,0.555732
1,vhGrzmvA3Hjq,0.427740,0.363022,0.402862,0.363003,0.534558,0.345347,0.409471,0.303328,0.402515,...,0.198009,-0.000724,0.083122,0.033043,0.687497,0.306229,0.717485,0.461809,0.559632,0.350027
2,ULliyEXjy4OV,0.139572,0.390106,-0.087041,0.196852,0.088148,0.023843,0.381782,0.068979,0.377488,...,0.051319,0.023630,-0.056819,0.117396,0.576086,0.517831,0.527044,0.605038,0.609856,0.750987
3,LZfeAb1xMtql,0.133561,0.778326,0.416355,0.471840,0.568460,0.633660,0.501113,0.345461,0.467943,...,0.046183,-0.238962,0.121868,-0.260970,0.646818,0.594902,0.608156,0.595459,0.683189,0.542296
4,EnFOUv0YK1RG,0.126699,0.575446,0.509422,0.363193,0.427544,0.449924,0.451796,0.223927,0.298248,...,0.315734,0.002234,0.290791,0.344149,0.480214,0.539824,0.447322,0.293088,0.148529,0.539823


## Data Exploration


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Columns: 19901 entries, participant_id to 198throw_199thcolumn
dtypes: float64(19900), object(1)
memory usage: 184.2+ MB


In [7]:
train_data.describe()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
count,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,...,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000,1213.000000
mean,0.060553,0.122315,0.060268,0.041287,0.069722,0.091007,0.066852,0.000252,0.014128,-0.002914,...,0.011075,-0.004938,-0.004378,0.001610,0.115171,0.049984,0.058144,0.093527,0.089403,0.128946
std,0.064178,0.054026,0.057495,0.043491,0.044222,0.049189,0.046864,0.049046,0.038205,0.042462,...,0.049632,0.046536,0.042900,0.047424,0.057128,0.051664,0.057674,0.054594,0.058036,0.058490
min,-0.183279,-0.059932,-0.145566,-0.127827,-0.072043,-0.079184,-0.105722,-0.164297,-0.137728,-0.148490,...,-0.161666,-0.176523,-0.178688,-0.138048,-0.070094,-0.153540,-0.131455,-0.085566,-0.204160,-0.083077
25%,0.018482,0.086102,0.026548,0.014457,0.042462,0.057614,0.036934,-0.031358,-0.010635,-0.030538,...,-0.021376,-0.033424,-0.033798,-0.030132,0.080291,0.015827,0.022316,0.059621,0.053224,0.090459
50%,0.058276,0.123220,0.061339,0.043246,0.067066,0.086494,0.067247,0.002549,0.016130,-0.002604,...,0.010254,-0.004683,-0.003724,0.000990,0.113640,0.052705,0.059151,0.093397,0.088612,0.127913
75%,0.100103,0.154518,0.099056,0.068408,0.096504,0.119404,0.095117,0.031053,0.038770,0.024507,...,0.044165,0.024913,0.024007,0.032268,0.150524,0.082526,0.095192,0.127144,0.127613,0.166523
max,0.321522,0.390895,0.278429,0.189825,0.317500,0.316811,0.270018,0.168196,0.145364,0.128301,...,0.194616,0.183152,0.180562,0.192015,0.375635,0.228748,0.322084,0.348153,0.267162,0.414304


In [8]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Columns: 19901 entries, participant_id to 198throw_199thcolumn
dtypes: float64(19900), object(1)
memory usage: 46.2+ MB


In [9]:
test_data.describe()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
count,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,...,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000,304.000000
mean,0.219970,0.610695,0.419458,0.395230,0.480795,0.530394,0.496573,0.287560,0.413862,0.244862,...,0.049241,-0.055612,-0.015886,0.022734,0.558894,0.353789,0.380367,0.503837,0.487597,0.557431
std,0.249959,0.163408,0.248612,0.244413,0.256404,0.167916,0.178327,0.201606,0.196153,0.218831,...,0.199660,0.225587,0.215424,0.206935,0.156629,0.203452,0.195547,0.170680,0.183347,0.173604
min,-0.713362,-0.118109,-0.376194,-0.379673,-0.379396,-0.022415,-0.113225,-0.516528,-0.195773,-0.555761,...,-0.363454,-0.501310,-0.570880,-0.479486,0.000000,-0.201091,-0.213887,-0.040523,-0.199617,-0.168106
25%,0.059497,0.511165,0.243170,0.267692,0.344056,0.424307,0.379608,0.159488,0.287149,0.089417,...,-0.093097,-0.227949,-0.164278,-0.128828,0.479489,0.235322,0.264928,0.401851,0.377687,0.456239
50%,0.219107,0.634942,0.453220,0.442026,0.539100,0.539635,0.524052,0.322734,0.426979,0.268674,...,0.037098,-0.077162,-0.014110,0.006236,0.577125,0.374062,0.404230,0.539822,0.502269,0.581648
75%,0.396008,0.727476,0.605324,0.561536,0.671823,0.652488,0.624549,0.429456,0.571971,0.411791,...,0.179494,0.100244,0.126558,0.154479,0.662010,0.508747,0.528825,0.627340,0.616772,0.682909
max,0.778206,0.904572,0.875906,0.915949,0.914515,0.920882,0.921036,0.887702,0.907231,0.877371,...,0.737060,0.681016,0.781968,0.809705,0.874170,0.780386,0.765132,0.859924,0.820123,0.872319


In [10]:
# check for duplicates in both training and testing data 
train_data.duplicated()


0       False
1       False
2       False
3       False
4       False
        ...  
1208    False
1209    False
1210    False
1211    False
1212    False
Length: 1213, dtype: bool

In [11]:
test_data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
299    False
300    False
301    False
302    False
303    False
Length: 304, dtype: bool

In [12]:
numeric_data = train_data.select_dtypes(include=['float64, int64'])

In [13]:
train_data.isnull().sum()

participant_id          0
0throw_1thcolumn        0
0throw_2thcolumn        0
0throw_3thcolumn        0
0throw_4thcolumn        0
                       ..
196throw_198thcolumn    0
196throw_199thcolumn    0
197throw_198thcolumn    0
197throw_199thcolumn    0
198throw_199thcolumn    0
Length: 19901, dtype: int64

In [14]:
# Check the data types of each column
non_numeric_columns = train_data.select_dtypes(exclude=[np.number]).columns
print("Non-numeric columns:", non_numeric_columns)


Non-numeric columns: Index(['participant_id'], dtype='object')


## Data Preprocessing 

In [ ]:
train_data.select_dtypes(include=[np.number]).corr()
# Normalizing the numeric features 
corr_matrix =train_data.drop(columns=['paticipant_id'])
plt.figure(figsize = (12,6))
sns.heatmap(corr_matrix, annot = True, cmap = 'coolwarm', fmt = '2f', vmin = -1, vmax= 1)
plt.title("Training Correlation")